## The Coupled Cluster Approximation

We can approximate the true wavefunction $\lvert \Psi \rangle$, of the system, by the coupled cluster wavefunction, $\lvert \Psi_{CC} \rangle$, defined by

$$
    \lvert \Psi_{CC} \rangle \equiv e^T \lvert \Phi_0\rangle
        \left(\sum_{i=0}^n \frac{1}{n!}T^n\right) \lvert \Phi_0\rangle
$$

But we have the complication that the cluster operator $T$ is

$$
    T = T_1 + T_2 + T_3 + \dots
$$

with

$$
    T_n = \left(\frac{1}{n!}\right)^2t^{i_1\dots i_n}_{a_1\dots a_n}a_1\dots a_n i_1 \dots i_n
$$

Use similarity-transformed Hamiltonian,
$$
    \mathfrak{H} = e^{-T} \hat{H}_Ne^{T}.
$$


Apply the Baker, the Camper and the House-Village,
$$
    \mathfrak{H} = \hat{H}_N + [\hat{H}_N, T] + \frac{1}{2!}[[\hat{H}_N, T], T] + \frac{1}{3!}[[[\hat{H}_N, T],T],T] + \frac{1}{4!}[[[[\hat{H}_N,T],T],T],T]
$$
and then it stops, because $\hat{H}_N$ has at most two-particle interactions.

In [2]:
from sympy.physics.secondquant import (AntiSymmetricTensor, wicks,
        F, Fd, NO, evaluate_deltas, substitute_dummies, Commutator,
        simplify_index_permutations, PermutationOperator)
from sympy import (
    symbols, expand, pprint, Rational, latex, Dummy
)

pretty_dummies_d = {
    'above': 'cdefgh',
    'below': 'klmn',
    'general': 'pqrst'
}

pretty_dummies_t = {
    'above': 'defgh',
    'below': 'klmn',
    'general': 'pqrs'
}

In [3]:
# Hamiltonian
def get_hamiltonian():
    p, q, r, s = symbols('p,q,r,s', cls=Dummy)
    f = AntiSymmetricTensor('f', (p,), (q,))
    pq = Fd(p)*F(q)
    u = AntiSymmetricTensor('u', (p,q), (r,s))
    pqrs = Fd(p)*Fd(q)*F(r)*F(s)
    H1 = f*NO(pq)
    H2 = Rational(1, 4)*u*NO(pqrs)
    return H1, H2

In [9]:
def compute_hausdorff(H, operator_function, pretty_dummies_dict):
    C = Commutator
    
    # I am not sure this works... 1+1+1+1+12+ekpqamfoinsz<0
    T = sum(operator_function())
    print("commutator 1...")
    comm1 = wicks(C(H, T))
    comm1 = evaluate_deltas(comm1)
    comm1 = substitute_dummies(comm1)

    T = sum(operator_function())
    print("commutator 2...")
    comm2 = wicks(C(comm1, T))
    comm2 = evaluate_deltas(comm2)
    comm2 = substitute_dummies(comm2)
    
    T = sum(operator_function())
    print("commutator 3...")
    comm3 = wicks(C(comm2, T))
    comm3 = evaluate_deltas(comm3)
    comm3 = substitute_dummies(comm3)

    T = sum(operator_function())
    print("commutator 4...")
    comm4 = wicks(C(comm3, T))
    comm4 = evaluate_deltas(comm4)
    comm4 = substitute_dummies(comm4)

    T = sum(operator_function())
    print("construct Hausdorff expansion...")
    eq = H + comm1 + comm2/2 + comm3/6 + comm4/24
    eq = eq.expand()
    eq = evaluate_deltas(eq)
    eq = substitute_dummies(eq, new_indices=True,
            pretty_indices=pretty_dummies_dict)
    
    print()
    return eq

In [11]:
# Cluster operators

# Singles
def get_singles_operator():
    i = symbols('i', below_fermi=True, cls=Dummy)
    a = symbols('a', above_fermi=True, cls=Dummy)
    t = AntiSymmetricTensor('t', (a,), (i,))
    ai = Fd(a)*F(i)
    T1 = t*NO(ai)
    return T1

# Doubles
def get_doubles_operator():
    i, j = symbols('i,j', below_fermi=True, cls=Dummy)
    a, b = symbols('a,b', above_fermi=True, cls=Dummy)
    t = AntiSymmetricTensor('t', (a, b), (i, j))
    abji = Fd(a)*Fd(b)*F(j)*F(i)
    T2 = Rational(1, 4)*t*NO(abji)
    return T2

# Triples
def get_triples_operator():
    i, j, k = symbols('i,j,k', below_fermi=True, cls=Dummy)
    a, b, c = symbols('a,b,c', above_fermi=True, cls=Dummy)
    t = AntiSymmetricTensor('t', (a, b, c), (i, j, k))
    abckji = Fd(a)*Fd(b)*Fd(c)*F(k)*F(j)*F(i)
    T3 = Rational(1, 36)*t*NO(abckji)
    return T3

## CCD Equations

In [17]:
# CCD equations
H1, H2 = get_hamiltonian()
equation = compute_hausdorff(H1+H2, lambda: [get_doubles_operator()],
                             pretty_dummies_d)
energy = wicks(equation, simplify_dummies=True, 
               keep_only_fully_contracted=True,
               simplify_kronecker_deltas=True)
energy = substitute_dummies(energy, new_indices=True, 
                            pretty_indices=pretty_dummies_d)
i, j = symbols('i,j', below_fermi=True)
a, b = symbols('a,b', above_fermi=True)
amplitude_equation = wicks(NO(Fd(i) * Fd(j) * F(b) * F(a)) * equation,
                             simplify_dummies=True,
                             keep_only_fully_contracted=True,
                             simplify_kronecker_deltas=True)
amplitude_equation = substitute_dummies(amplitude_equation,
                                       new_indices=True,
                                       pretty_indices=pretty_dummies_d)

commutator 1...
commutator 2...
commutator 3...
commutator 4...
construct Hausdorff expansion...



In [18]:
print("E = ", latex(energy), "\n")
print("(doubles) 0 = ", latex(amplitude_equation))

E =  - \frac{t^{dc}_{lk} u^{lk}_{dc}}{4} 

(doubles) 0 =  f^{k}_{i} t^{ab}_{jk} - f^{k}_{j} t^{ab}_{ik} - f^{a}_{c} t^{bc}_{ij} + f^{b}_{c} t^{ac}_{ij} - \frac{t^{dc}_{ik} t^{ab}_{jl}}{2} u^{lk}_{dc} - \frac{t^{dc}_{ij} t^{ab}_{lk}}{4} u^{lk}_{dc} - \frac{t^{dc}_{ij} u^{ab}_{dc}}{2} + \frac{t^{dc}_{jk} t^{ab}_{il}}{2} u^{lk}_{dc} - \frac{t^{ac}_{lk} t^{bd}_{ij}}{2} u^{lk}_{dc} - t^{ac}_{ik} t^{bd}_{jl} u^{lk}_{dc} - t^{ac}_{ik} u^{bk}_{jc} - \frac{t^{ac}_{ij} t^{bd}_{lk}}{2} u^{lk}_{dc} + t^{ac}_{jk} t^{bd}_{il} u^{lk}_{dc} + t^{ac}_{jk} u^{bk}_{ic} - \frac{t^{ab}_{lk} u^{lk}_{ij}}{2} + t^{bc}_{ik} u^{ak}_{jc} - t^{bc}_{jk} u^{ak}_{ic} - u^{ab}_{ij}


CCD energy
$$
    - \frac{t^{dc}_{lk} u^{lk}_{dc}}{4}
$$
CCD amplitude equation
$$
   0 =  f^{k}_{i} t^{ab}_{jk} - f^{k}_{j} t^{ab}_{ik} - f^{a}_{c} t^{bc}_{ij} + f^{b}_{c} t^{ac}_{ij} - \frac{t^{dc}_{ik} t^{ab}_{jl}}{2} u^{lk}_{dc} - \frac{t^{dc}_{ij} t^{ab}_{lk}}{4} u^{lk}_{dc} - \frac{t^{dc}_{ij} u^{ab}_{dc}}{2} + \frac{t^{dc}_{jk} t^{ab}_{il}}{2} u^{lk}_{dc} - \frac{t^{ac}_{lk} t^{bd}_{ij}}{2} u^{lk}_{dc} - t^{ac}_{ik} t^{bd}_{jl} u^{lk}_{dc} - t^{ac}_{ik} u^{bk}_{jc} - \frac{t^{ac}_{ij} t^{bd}_{lk}}{2} u^{lk}_{dc} + t^{ac}_{jk} t^{bd}_{il} u^{lk}_{dc} + t^{ac}_{jk} u^{bk}_{ic} - \frac{t^{ab}_{lk} u^{lk}_{ij}}{2} + t^{bc}_{ik} u^{ak}_{jc} - t^{bc}_{jk} u^{ak}_{ic} - u^{ab}_{ij}
$$

## CCDT Equations

In [20]:
# CCDT equations
H1, H2 = get_hamiltonian()
equation = compute_hausdorff(H1+H2,
                             lambda: [
                                 get_doubles_operator(),
                                 get_triples_operator()
                             ],
                             pretty_dummies_t)
energy = wicks(equation, simplify_dummies=True, 
               keep_only_fully_contracted=True,
               simplify_kronecker_deltas=True)
energy = substitute_dummies(energy, new_indices=True, 
                            pretty_indices=pretty_dummies_d) 
i, j = symbols('i,j', below_fermi=True)
a, b = symbols('a,b', above_fermi=True)
doubles_amplitude = wicks(NO(Fd(i) * Fd(j) * F(b) * F(a)) * equation,
                             simplify_dummies=True,
                             keep_only_fully_contracted=True,
                             simplify_kronecker_deltas=True)
doubles_amplitude = substitute_dummies(doubles_amplitude,
                                      new_indices=True,
                                      pretty_indices=pretty_dummies_t)
i, j, k = symbols('i,j,k', below_fermi=True)
a, b, c = symbols('a,b,c', above_fermi=True)
triples_amplitude = wicks(NO(Fd(i) * Fd(j) * Fd(k) * F(c) * F(b) * F(a)) * equation,
                         simplify_dummies=True,
                         keep_only_fully_contracted=True,
                         simplify_kronecker_deltas=True)
triples_amplitude = simplify_index_permutations(triples_amplitude,
                            [
                                PermutationOperator(a, b),
                                PermutationOperator(a, c),
                                PermutationOperator(b, c),
                                PermutationOperator(i, j),
                                PermutationOperator(i, k),
                                PermutationOperator(j, k)
                            ])
triples_amplitude = substitute_dummies(triples_amplitude,
                                      new_indices=True,
                                      pretty_indices=pretty_dummies_t)

commutator 1...
commutator 2...
commutator 3...
commutator 4...
construct Hausdorff expansion...



In [22]:
print("E = ", latex(energy), "\n")
print("(doubles) 0 = ", latex(doubles_amplitude), "\n")
print("(triples) 0 = ", latex(triples_amplitude))

E =  - \frac{t^{dc}_{lk} u^{lk}_{dc}}{4} 

(doubles) 0 =  f^{k}_{d} t^{abd}_{ijk} + f^{k}_{i} t^{ab}_{jk} - f^{k}_{j} t^{ab}_{ik} - f^{a}_{d} t^{bd}_{ij} + f^{b}_{d} t^{ad}_{ij} - \frac{t^{ed}_{ik} t^{ab}_{jl}}{2} u^{lk}_{ed} - \frac{t^{ed}_{ij} t^{ab}_{lk}}{4} u^{lk}_{ed} - \frac{t^{ed}_{ij} u^{ab}_{ed}}{2} + \frac{t^{ed}_{jk} t^{ab}_{il}}{2} u^{lk}_{ed} - \frac{t^{ad}_{lk} t^{be}_{ij}}{2} u^{lk}_{ed} - t^{ad}_{ik} t^{be}_{jl} u^{lk}_{ed} - t^{ad}_{ik} u^{bk}_{jd} - \frac{t^{ad}_{ij} t^{be}_{lk}}{2} u^{lk}_{ed} + t^{ad}_{jk} t^{be}_{il} u^{lk}_{ed} + t^{ad}_{jk} u^{bk}_{id} - \frac{t^{ab}_{lk} u^{lk}_{ij}}{2} + t^{bd}_{ik} u^{ak}_{jd} - t^{bd}_{jk} u^{ak}_{id} - \frac{t^{aed}_{ijk} u^{bk}_{ed}}{2} + \frac{t^{abd}_{ilk} u^{lk}_{jd}}{2} - \frac{t^{abd}_{jlk} u^{lk}_{id}}{2} + \frac{t^{bed}_{ijk} u^{ak}_{ed}}{2} - u^{ab}_{ij} 

(triples) 0 =  - f^{k}_{d} t^{ad}_{ij} t^{cb}_{kk} P(ab) - f^{k}_{d} t^{ab}_{ik} t^{cd}_{kj} P(ij) + f^{k}_{d} t^{ab}_{kk} t^{cd}_{ij} + f^{k}_{d} t^{ac}_{ik} t^{

Praise science! And also; holy sh*t